# List Comprehension Syntax

Pythonistas love this syntax, because it is compact and expressive i.e. "pythonic".

The idea is a for loop between square brackets generates the terms of the list.

```
    [<expression using var> for var in iterable]
```

is the core syntax.

Below we have chosen ```x``` to iterate over ```range(10)``` thereby binding the  name ```x``` to 0, 1, 2... 9.  Each time ```x``` binds to an object (in this case a number), the expression using ```x```, which is ```x * x```, computes the next term in the list.

In [ ]:
L = [x * x for x in range(10)]

In [ ]:
L

In the code cell below, practice variations on ```[1]``` by:

* changing the variable name
* altering the expression
* changing the iterable (e.g. try a new range or maybe another list)

An optional if clause allows for filtering:

```
       [  <expression using var> 
       for var in iterable
       if <expression using var>  ]
```

Splitting a list comprehension across multiple lines may add readability but is never required.

In [ ]:
words = ['tower', 'harry', 'potter', 'python']
M = [word.capitalize() 
     for word in words 
     if "p" in word]

In [ ]:
M

Practice makes perfect.  Consider adding some slice notation on the end, if you wish to practice a syntax we encountered earlier.

In [ ]:
[w.capitalize() for w in ['hello', 'kitty']][::-1]  # adding slice notation

Sometimes the expression does not need to involve any input from the for loop. Perhaps the new values are coming from someplace else, or maybe the expression is something static (unchanging).

Use of the name ```_``` typically signifies the values it takes on (binds to) have no relevance to the work at hand.  The point was simply to trigger repetition.  The for loop still runs multiple times, and sometimes that's all we need.

In [ ]:
stars = [ "*" for _ in range(10)]

In [ ]:
stars

As our segue to a next topic, the generator function, check out the cells below.  Your instructor will explain, as we turn to this related aspect of Python.

In [ ]:
# Fibonacci Numbers
# https://oeis.org/A000045

def fibo(a=0, b=1):
    while True:
        yield a
        b, a = a + b, b
        
gen = fibo()
fibs = [next(gen) for _ in range(10)]
print(fibs)

In [ ]:
fibs_gen = (next(gen) for _ in range(10))  # generator expression
type(fibs_gen)

In [ ]:
next(fibs_gen)

In [ ]:
next(fibs_gen)